<a href="https://colab.research.google.com/github/myqbit/or-tools/blob/stable/Cpk_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Part 1: Number generator function. Do not edit**

In [1]:
import pandas as pd
import numpy as np

def generate_random_numbers(data):
    while True:
        # Extract the desired parameters from the imported data
        mean = data['Mean']
        lower_bound = data['Lower Bound']
        upper_bound = data['Upper Bound']
        cpk = data['Cpk']

        # Calculate the required standard deviation
        std = min((mean - lower_bound) / (3 * cpk), (upper_bound - mean) / (3 * cpk))

        # Generate random numbers following a normal distribution
        numbers = np.random.normal(mean, std, 30)

        # Truncate the numbers to the desired range
        truncated_numbers = np.clip(numbers, lower_bound, upper_bound)

        # Add noise to each number
        noise_factor = 0.02  # Adjust the noise factor as desired
        noisy_numbers = truncated_numbers + np.random.uniform(-noise_factor, noise_factor, 30)

        # Calculate the actual mean and Cpk of the generated numbers
        actual_mean = np.mean(noisy_numbers)
        actual_std = np.std(noisy_numbers, ddof=1)  # Use ddof=1 for sample standard deviation

        if lower_bound == 0:
            actual_cpk = (upper_bound - actual_mean) / (3 * actual_std)
        else:
            actual_cpk = min((upper_bound - actual_mean) / (3 * actual_std), (actual_mean - lower_bound) / (3 * actual_std))

        if actual_cpk >= cpk:
            break

    # Round the numbers to three decimal places
    rounded_numbers = np.round(noisy_numbers, 3)
    gen_num = rounded_numbers.tolist()
    gen_num.append(actual_cpk)

    return gen_num




**Part 2: Excel file locator**

In [2]:

# Raw string with the file path
# Change the path to your excel file
excel_file_path = r'C:\Users\acryl\OneDrive\Desktop\Python\cpk_data.xlsx'



**Part 3: Main program body. Do not edit**

In [3]:
# Load data from Excel file
data = pd.read_excel(excel_file_path)
df = pd.DataFrame(data)
print(df)

# Create empty lists to store generated numbers and actual Cpk values
generated_numbers = []
actual_cpk_values = []

for index, row in df.iterrows():
    dict_cpk = {}
    dict_cpk["Mean"] = row['Mean']
    dict_cpk["Lower Bound"] = row["Lower Bound"]
    dict_cpk["Upper Bound"] = row["Upper Bound"]
    dict_cpk["Cpk"] = row["Cpk"]
    random_numbers = generate_random_numbers(dict_cpk)
    generated_numbers.append(random_numbers[:-1])  # Exclude the actual Cpk value from the list
    actual_cpk_values.append(random_numbers[-1])  # Get the actual Cpk value

# Add the generated numbers and actual Cpk values as new columns to the DataFrame
df['Generated Number'] = generated_numbers
df['Actual Cpk'] = actual_cpk_values

# Break generated numbers to individual column
for index in range(len(generated_numbers)):
    for sub_index in range(30):
        sub1 = sub_index + 1
        df.at[index, sub1] = generated_numbers[index][sub_index]

print(df)

# Write the updated DataFrame back to the Excel file
df.to_excel(excel_file_path, index=False)

FileNotFoundError: ignored